In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505212 sha256=dc3ba9cc6db672f86bf2e10626722ec95a3539be54543d2c7dae6a9d45c1853b
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
import numpy as np
import pandas as pd

In [4]:
netflix_data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/combinedNetflixData.txt', header = None, names =['Cust_id', 'Rating'], usecols = [0,1])

netflix_data

,Cust_id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
24058258,2591364,2.0
24058259,1791000,2.0
24058260,512536,5.0
24058261,988963,3.0


In [5]:
netflix_data.iloc[0:548]

,Cust_id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0
...,...,...
543,1790158,4.0
544,1403184,3.0
545,1535440,4.0
546,1426604,4.0


In [6]:
#how many movies are there in the dataset
movie_count = netflix_data.isnull().sum().iloc[1]

movie_count

np.int64(4499)

In [7]:
netflix_data[netflix_data["Rating"].isnull()]

,Cust_id,Rating
0,1:,NaN
548,2:,NaN
694,3:,NaN
2707,4:,NaN
2850,5:,NaN
...,...,...
24046714,4495:,NaN
24047329,4496:,NaN
24056849,4497:,NaN
24057564,4498:,NaN


In [8]:
netflix_data['Cust_id'].nunique()

475257

In [9]:
netflix_data['Cust_id'].nunique()-movie_count

np.int64(470758)

In [10]:
netflix_data["Rating"].value_counts()

,count
Rating,
4.0,8085741
3.0,6904181
5.0,5506583
2.0,2439073
1.0,1118186


In [11]:
#to remove movie ids from the customer id column

current_id = None
movie_ids = []

for id in netflix_data["Cust_id"]:
  if ':' in id:
    current_id = int(id.replace(':',''))
  movie_ids.append(current_id)

netflix_data["Movie_id"] = movie_ids

In [12]:
netflix_data

,Cust_id,Rating,Movie_id
0,1:,NaN,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
...,...,...,...
24058258,2591364,2.0,4499
24058259,1791000,2.0,4499
24058260,512536,5.0,4499
24058261,988963,3.0,4499


In [13]:
netflix_data.dropna(inplace = True)

In [14]:
netflix_data

,Cust_id,Rating,Movie_id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1
...,...,...,...
24058258,2591364,2.0,4499
24058259,1791000,2.0,4499
24058260,512536,5.0,4499
24058261,988963,3.0,4499


In [15]:
netflix_data = netflix_data.reset_index().drop('index',axis = 1)

netflix_data

,Cust_id,Rating,Movie_id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1
...,...,...,...
24053759,2591364,2.0,4499
24053760,1791000,2.0,4499
24053761,512536,5.0,4499
24053762,988963,3.0,4499


In [16]:
#to get the count and average of rating and grouping on movie ids

f = ['count','mean']

ratings_summary = netflix_data.groupby('Movie_id')['Rating'].agg(f).reset_index().sort_values('count')

ratings_summary

,Movie_id,count,mean
4361,4362,36,2.000000
4337,4338,39,2.948718
3655,3656,42,3.214286
914,915,43,2.837209
4293,4294,44,4.272727
...,...,...,...
570,571,154832,3.962585
4431,4432,156183,3.755364
3859,3860,160454,3.434517
2151,2152,162597,3.425322


In [17]:
#to remove low count ratings

benchmark = ratings_summary['count'].quantile(0.7)

benchmark = round(benchmark)

benchmark

1799

In [18]:
drop_list = ratings_summary[ratings_summary['count'] < benchmark].index

drop_list

Index([4361, 4337, 3655,  914, 4293,  819, 2804, 3692, 4034, 1857,
       ...
       2276, 2949, 2523, 3808, 1154, 3997, 2092, 2058, 1104, 1940],
      dtype='int64', length=3149)

In [19]:
final_df=netflix_data[~netflix_data['Movie_id'].isin(drop_list)]

final_df

,Cust_id,Rating,Movie_id
547,2059652,4.0,2
548,1666394,3.0,2
549,1759415,4.0,2
550,1959936,5.0,2
551,998862,4.0,2
...,...,...,...
24053759,2591364,2.0,4499
24053760,1791000,2.0,4499
24053761,512536,5.0,4499
24053762,988963,3.0,4499


In [20]:
title = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/movies.csv')

title

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


In [21]:
pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [23]:
import math
import seaborn as sns
import numpy as np

from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [24]:
#loading the reader library

reader = Reader()

NameError: name 'Reader' is not defined

In [ ]:
data = Dataset.load_from_df(final_df[['Cust_id','Rating','Movie_id']][:100000],reader)

In [ ]:
svd = SVD()

In [ ]:
data-100%(5 folds)
a b c d e

#80 train 20% test
iter1:
a,b,c,d-train
e-test
#rmse
iter2:
a,b,c,e-train
d-train

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-25-162908331684>, line 1)

In [ ]:
cross_validate(svd,data,measures=['RMSE','MAE'], verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    77.7320 78.0552 77.7862 77.7860 77.5114 77.7741 0.1733  
MAE (testset)     67.4530 67.7581 67.5335 67.5069 67.2733 67.5049 0.1557  
Fit time          1.66    1.76    2.79    1.75    1.68    1.93    0.43    
Test time         0.13    0.18    0.11    0.41    0.39    0.25    0.13    


{'test_rmse': array([77.73197669, 78.05516319, 77.78619479, 77.78603249, 77.51137465]),
 'test_mae': array([67.453  , 67.7581 , 67.5335 , 67.50685, 67.2733 ]),
 'fit_time': (1.6573395729064941,
  1.7555277347564697,
  2.787790536880493,
  1.7464179992675781,
  1.6818923950195312),
 'test_time': (0.13270950317382812,
  0.18483638763427734,
  0.11170268058776855,
  0.4126393795013428,
  0.3889279365539551)}

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7017289 entries, 547 to 24053763
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_id   object 
 1   Rating    float64
 2   Movie_id  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 214.2+ MB


In [ ]:
final_df['Cust_id'] = final_df['Cust_id'].astype(int)

<ipython-input-28-99ca9d615ea2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Cust_id'] = final_df['Cust_id'].astype(int)


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7017289 entries, 547 to 24053763
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_id   int64  
 1   Rating    float64
 2   Movie_id  int64  
dtypes: float64(1), int64(2)
memory usage: 214.2 MB


In [ ]:
np.random.choice(final_df['Cust_id'])

1116080

In [ ]:
user_1116080 = final_df[(final_df['Cust_id'] == 1116080)&(final_df['Rating'] == 5)]

user_1116080 = user_1116080.set_index('Movie_id')

user_1116080

,Cust_id,Rating
Movie_id,,
239,1116080,5.0
329,1116080,5.0
760,1116080,5.0
788,1116080,5.0
906,1116080,5.0
985,1116080,5.0
1110,1116080,5.0
1144,1116080,5.0
1173,1116080,5.0


In [ ]:
title = title.rename(columns = {'movieId':'Movie_id','title':'title','genre':'genre'})

title

,Movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


In [ ]:
pd.merge(user_1116080,title, on = 'Movie_id')

,Movie_id,Cust_id,Rating,title,genres
0,239,1116080,5.0,"Goofy Movie, A (1995)",Animation|Children|Comedy|Romance
1,329,1116080,5.0,Star Trek: Generations (1994),Adventure|Drama|Sci-Fi
2,760,1116080,5.0,Stalingrad (1993),Drama|War
3,788,1116080,5.0,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi
4,906,1116080,5.0,Gaslight (1944),Drama|Thriller
5,985,1116080,5.0,Small Wonders (1995),Documentary
6,1110,1116080,5.0,Bird of Prey (1996),Action
7,1144,1116080,5.0,"Line King: The Al Hirschfeld Story, The (1996)",Documentary
8,1173,1116080,5.0,"Cook the Thief His Wife & Her Lover, The (1989)",Comedy|Drama
9,1255,1116080,5.0,Bad Taste (1987),Comedy|Horror|Sci-Fi


In [ ]:
user_1116080 = title

user_1116080

,Movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


In [ ]:
user_1116080 = user_1116080[user_1116080['Movie_id'].isin(drop_list)]

user_1116080

,Movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
4389,4493,Critters 2: The Main Course (1988),Comedy|Horror|Sci-Fi
4390,4494,Critters 3 (1991),Comedy|Sci-Fi
4392,4496,D.O.A. (1988),Film-Noir|Mystery|Thriller
4393,4497,Dead Heat (1988),Action|Comedy|Horror|Sci-Fi


In [ ]:
#Prediction of rating for user

user_1116080['estimation report'] = user_1116080['Movie_id'].apply(lambda x: svd.predict(1116080,x).est)

<ipython-input-44-c743529f2959>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_1116080['estimation report'] = user_1116080['Movie_id'].apply(lambda x: svd.predict(1116080,x).est)


In [ ]:
user_1116080.sort_values('estimation report')

user_1116080.head(10)

,Movie_id,title,genres,estimation report
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
2,3,Grumpier Old Men (1995),Comedy|Romance,5
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,5
4,5,Father of the Bride Part II (1995),Comedy,5
5,6,Heat (1995),Action|Crime|Thriller,5
7,8,Tom and Huck (1995),Adventure|Children,5
8,9,Sudden Death (1995),Action,5
9,10,GoldenEye (1995),Action|Adventure|Thriller,5
10,11,"American President, The (1995)",Comedy|Drama|Romance,5
11,12,Dracula: Dead and Loving It (1995),Comedy|Horror,5
